# Notebook 6: Production Pipeline - Full IARS Integration

**Objective**: Deploy a complete Intelligent Approval Routing System:
- Integrate all Cleanlab functions via Datalab
- Implement temperature-aware routing with annealing
- Handle multiple collapse types simultaneously
- Production monitoring and metrics

---

## Flow Diagram

```mermaid
flowchart TD
    subgraph Input["📥 Approval Request"]
        A[Request Text]
        B[Amount/Category]
        C[Requestor Info]
    end

    subgraph Embedding["🔤 Feature Extraction"]
        D[Text Embedding]
        E[Structured Features]
        F[Combined Vector]
    end

    subgraph Classifier["🤖 ML Classification"]
        G[Category Classifier]
        H[Prediction Probs]
        I[Confidence Score]
    end

    subgraph Datalab["🧹 Cleanlab Datalab"]
        J[Datalab.find_issues]
        K[Label Quality]
        L[OOD Score]
        M[Near Duplicates]
        N[Dataset Health]
    end

    subgraph YRSN["🎯 YRSN Decomposition"]
        O[CleanlabAdapter]
        P[R Component]
        Q[S Component]
        R2[N Component]
        S2[Collapse Detection]
    end

    subgraph Urgency["⏰ Urgency Scoring"]
        T[T_expiry]
        U[P_business]
        V[S_sentiment]
        W[C_clarity]
        X[Weighted Sum]
    end

    subgraph Temperature["🌡️ Temperature Routing"]
        Y[Compute τ = 1/α]
        Z[Annealing Schedule]
        AA[Threshold Adjustment]
    end

    subgraph Routing["🚦 Stream Decision"]
        AB{Knockout Rules?}
        AC{Collapse Type?}
        AD{Confidence + YRSN?}
        AE[🟢 GREEN: Auto-process]
        AF[🟡 YELLOW: AI-assisted]
        AG[🔴 RED: Expert review]
    end

    subgraph Output["📤 Decision"]
        AH[RoutingDecision]
        AI[Soft Probabilities]
        AJ[Urgency Level]
        AK[Audit Trail]
    end

    subgraph Monitoring["📊 Production Metrics"]
        AL[Automation Rate]
        AM[Temperature Distribution]
        AN[Collapse Frequency]
        AO[Quality Trends]
    end

    A --> D
    B --> E
    C --> E
    D --> F
    E --> F
    F --> G
    G --> H
    H --> I
    H --> J
    J --> K
    J --> L
    J --> M
    J --> N
    K --> O
    L --> O
    M --> O
    O --> P
    O --> Q
    O --> R2
    O --> S2
    C --> T
    B --> U
    A --> V
    P --> W
    R2 --> W
    T --> X
    U --> X
    V --> X
    W --> X
    P --> Y
    Y --> Z
    Z --> AA
    I --> AB
    AB -->|Yes| AG
    AB -->|No| AC
    S2 --> AC
    AC -->|POISONING/CONFUSION| AG
    AC -->|DISTRACTION| AF
    AC -->|NONE/CLASH| AD
    AA --> AD
    AD -->|High Cs, Low τ| AE
    AD -->|Medium| AF
    AD -->|Low Cs, High τ| AG
    AE --> AH
    AF --> AH
    AG --> AH
    AH --> AI
    X --> AJ
    AH --> AK
    AH --> AL
    Y --> AM
    S2 --> AN
    N --> AO

    style Datalab fill:#e1f5fe
    style YRSN fill:#e8f5e9
    style Temperature fill:#fff3e0
    style Routing fill:#fce4ec
    style Monitoring fill:#f3e5f5
```

---

**Collapse Types Handled**: ALL (POISONING, DISTRACTION, CONFUSION, CLASH)

**Difficulty**: ⭐⭐⭐ Hard

## 1. Setup

In [ ]:
# Install dependencies
!pip install cleanlab sentence-transformers scikit-learn pandas numpy --quiet

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sentence_transformers import SentenceTransformer

# Cleanlab Datalab (unified interface)
from cleanlab import Datalab
from cleanlab.dataset import (
    overall_label_health_score,
    rank_classes_by_label_quality,
    find_overlapping_classes
)
from cleanlab.filter import get_confident_thresholds

# Import YRSN-IARS modules
import sys
sys.path.append('../src')
from yrsn_iars.adapters.cleanlab_adapter import CleanlabAdapter, YRSNResult, CollapseType
from yrsn_iars.adapters.temperature import (
    TemperatureScheduler, TemperatureConfig, TemperatureMode,
    AdaptiveRoutingEngine, compute_temperature
)
from yrsn_iars.pipelines.approval_router import (
    ApprovalRouter, ApprovalRequest, RoutingDecision, 
    Stream, UrgencyLevel, UrgencyWeights
)

print("All dependencies loaded successfully")

## 2. Generate Production-Scale Dataset

In [ ]:
np.random.seed(42)
n_samples = 2000

# Realistic approval categories
categories = [
    'software_license', 'travel_request', 'budget_variance',
    'vendor_payment', 'equipment_purchase', 'training_expense',
    'contractor_invoice', 'marketing_spend', 'office_supplies',
    'professional_services'
]

# Generate diverse request data
data = []
for i in range(n_samples):
    category = np.random.choice(categories)
    
    # Amount distribution varies by category
    if category in ['budget_variance', 'vendor_payment']:
        amount = np.random.exponential(50000)
    elif category in ['software_license', 'professional_services']:
        amount = np.random.exponential(15000)
    elif category == 'office_supplies':
        amount = np.random.exponential(500)
    else:
        amount = np.random.exponential(5000)
    
    amount = max(100, min(500000, amount))  # Clamp
    
    # Text templates
    templates = {
        'software_license': f"Request for {np.random.choice(['AWS', 'Azure', 'Salesforce', 'Adobe', 'Slack'])} license - ${amount:,.0f}",
        'travel_request': f"Travel to {np.random.choice(['NYC', 'SF', 'London', 'Tokyo', 'Dubai'])} for {np.random.choice(['client meeting', 'conference', 'training'])}",
        'budget_variance': f"Q{np.random.randint(1,5)} budget adjustment of ${amount:,.0f} for {np.random.choice(['project overrun', 'new initiative', 'unforeseen costs'])}",
        'vendor_payment': f"Payment to {np.random.choice(['Acme Corp', 'Tech Solutions', 'Global Services'])} - Invoice #{np.random.randint(10000, 99999)}",
        'equipment_purchase': f"Purchase of {np.random.choice(['laptops', 'monitors', 'servers', 'network equipment'])} for {np.random.choice(['new hires', 'refresh', 'expansion'])}",
        'training_expense': f"{np.random.choice(['AWS Certification', 'Leadership Training', 'Technical Workshop'])} for {np.random.randint(1, 20)} employees",
        'contractor_invoice': f"Contractor {np.random.choice(['development', 'consulting', 'design'])} services - {np.random.randint(40, 200)} hours",
        'marketing_spend': f"{np.random.choice(['Digital ads', 'Trade show', 'Content creation'])} campaign Q{np.random.randint(1,5)}",
        'office_supplies': f"Office supplies order - {np.random.choice(['paper', 'printer ink', 'ergonomic accessories'])}",
        'professional_services': f"{np.random.choice(['Legal', 'Accounting', 'HR Consulting'])} services engagement"
    }
    
    # Decision (with some noise)
    # Base approval rate varies by category
    base_rates = {
        'office_supplies': 0.95,
        'training_expense': 0.85,
        'software_license': 0.80,
        'travel_request': 0.75,
        'equipment_purchase': 0.70,
        'marketing_spend': 0.65,
        'contractor_invoice': 0.70,
        'professional_services': 0.65,
        'vendor_payment': 0.80,
        'budget_variance': 0.50
    }
    
    # Amount affects approval
    amount_modifier = -0.1 * (amount / 100000)  # Higher amounts less likely approved
    approval_prob = base_rates[category] + amount_modifier
    decision = 1 if np.random.random() < approval_prob else 0
    
    # Deadline (some urgent, some not)
    if np.random.random() < 0.2:
        deadline = datetime.now() + timedelta(hours=np.random.randint(2, 48))
    elif np.random.random() < 0.5:
        deadline = datetime.now() + timedelta(days=np.random.randint(1, 14))
    else:
        deadline = None
    
    data.append({
        'request_id': f'REQ-{i:05d}',
        'text': templates[category],
        'category': category,
        'amount': amount,
        'decision': decision,
        'deadline': deadline,
        'requestor_level': np.random.randint(1, 8),
        'requestor_id': f'EMP-{np.random.randint(1000, 9999)}'
    })

df = pd.DataFrame(data)

# Inject label noise (realistic)
noise_rate = 0.08
noise_indices = np.random.choice(n_samples, size=int(n_samples * noise_rate), replace=False)
df.loc[noise_indices, 'decision'] = 1 - df.loc[noise_indices, 'decision']

# Inject some near-duplicates
for _ in range(50):
    source_idx = np.random.randint(0, n_samples)
    duplicate = df.iloc[source_idx].copy()
    duplicate['request_id'] = f'REQ-DUP-{np.random.randint(10000, 99999)}'
    duplicate['text'] = duplicate['text'] + " (duplicate request)"
    df = pd.concat([df, pd.DataFrame([duplicate])], ignore_index=True)

print(f"Generated {len(df)} approval requests")
print(f"\nCategory distribution:")
print(df['category'].value_counts())
print(f"\nDecision distribution: {df['decision'].value_counts().to_dict()}")

## 3. Feature Engineering & Classifier Training

In [ ]:
# Generate text embeddings
print("Loading embedding model...")
encoder = SentenceTransformer('all-MiniLM-L6-v2')

print("Generating embeddings...")
text_embeddings = encoder.encode(df['text'].tolist(), show_progress_bar=True)

# Structured features
scaler = StandardScaler()
structured_features = scaler.fit_transform(
    df[['amount', 'requestor_level']].values
)

# Combine features
X = np.hstack([text_embeddings, structured_features])

# Encode labels
y = df['decision'].values

print(f"Feature matrix shape: {X.shape}")

In [ ]:
# Train classifier with cross-validation
print("Training classifier...")
clf = LogisticRegression(max_iter=1000, n_jobs=-1)

pred_probs = cross_val_predict(
    clf,
    X, y,
    cv=5,
    method='predict_proba',
    n_jobs=-1
)

# Train final model for inference
clf.fit(X, y)

print(f"Cross-validation complete")
print(f"Prediction shape: {pred_probs.shape}")

## 4. Cleanlab Datalab Analysis

In [ ]:
# Create Datalab for unified issue detection
datalab_data = {
    'text': df['text'].tolist(),
    'label': df['decision'].tolist()
}

print("Running Datalab analysis...")
lab = Datalab(data=datalab_data, label_name='label')

# Find all issues
lab.find_issues(
    pred_probs=pred_probs,
    features=text_embeddings  # For outlier/duplicate detection
)

print("\nDatalab Issue Summary:")
lab.report()

In [ ]:
# Extract all Datalab signals
issues_df = lab.get_issues()

print(f"Datalab columns: {issues_df.columns.tolist()}")
print(f"\nIssue statistics:")
for col in issues_df.columns:
    if col.startswith('is_'):
        print(f"  {col}: {issues_df[col].sum()} issues")

In [ ]:
# Dataset-level health
health_score = overall_label_health_score(y, pred_probs)
print(f"\nOverall Dataset Health Score: {health_score:.3f}")

# Class quality
class_quality = rank_classes_by_label_quality(
    labels=y,
    pred_probs=pred_probs,
    class_names=['Reject', 'Approve']
)
print(f"\nClass Quality:")
print(class_quality)

# Confident thresholds
thresholds = get_confident_thresholds(y, pred_probs)
print(f"\nConfident Thresholds: {thresholds}")

## 5. YRSN Decomposition via Adapter

In [ ]:
# Initialize adapter
adapter = CleanlabAdapter()

# Convert Datalab results to YRSN
yrsn_df = adapter.from_datalab(lab, include_details=True)

# Merge with original data
df = df.reset_index(drop=True)
df = pd.concat([df, yrsn_df[['R', 'S', 'N', 'quality_score', 'risk_score', 'collapse_type']]], axis=1)

print("YRSN Statistics:")
print(df[['R', 'S', 'N']].describe())

print(f"\nCollapse Type Distribution:")
print(df['collapse_type'].value_counts())

## 6. Initialize Production Router with Annealing

In [ ]:
# Configure temperature with annealing (production warm-up)
temp_config = TemperatureConfig(
    mode=TemperatureMode.ANNEALING,
    tau_initial=2.0,      # Start loose (more human review)
    tau_final=0.5,        # End tight (more automation)
    annealing_steps=500,  # Steps to full automation
    tau_min=0.3,
    tau_max=3.0
)

# Initialize router
router = ApprovalRouter(
    cleanlab_adapter=adapter,
    temperature_config=temp_config,
    knockout_rules=[
        {
            "rule": "amount_exceeds_limit",
            "condition": lambda r: r.amount > 250000,
            "message": "Amount exceeds $250K limit",
            "action": "ROUTE_TO_EXPERT"
        },
        {
            "rule": "auto_approve_supplies",
            "condition": lambda r: r.amount < 200 and r.category == "office_supplies",
            "message": "Below auto-approval threshold",
            "action": "APPROVE"
        }
    ]
)

print("Router initialized with ANNEALING temperature schedule")
print(f"Initial τ: {temp_config.tau_initial}, Final τ: {temp_config.tau_final}")

## 7. Process All Requests Through Router

In [ ]:
# Process each request
decisions = []

for idx, row in df.iterrows():
    # Create ApprovalRequest
    request = ApprovalRequest(
        request_id=row['request_id'],
        text=row['text'],
        category=row['category'],
        amount=row['amount'],
        requestor_id=row['requestor_id'],
        deadline=row['deadline'],
        requestor_level=row['requestor_level']
    )
    
    # Get label quality from Datalab
    label_quality = issues_df.loc[idx, 'label_score'] if 'label_score' in issues_df.columns else 0.8
    
    # Get OOD score
    ood_score = 1 - issues_df.loc[idx, 'outlier_score'] if 'outlier_score' in issues_df.columns else 0.9
    
    # Get duplicate flag
    is_duplicate = issues_df.loc[idx, 'is_near_duplicate_issue'] if 'is_near_duplicate_issue' in issues_df.columns else False
    
    # Classifier confidence
    classifier_conf = pred_probs[idx].max()
    
    # Route request
    decision = router.route(
        request=request,
        classifier_confidence=classifier_conf,
        label_quality=label_quality,
        pred_probs=pred_probs[idx],
        ood_score=ood_score,
        is_duplicate=is_duplicate
    )
    
    decisions.append(decision)
    
    # Progress
    if (idx + 1) % 500 == 0:
        print(f"Processed {idx + 1}/{len(df)} requests")

print(f"\nRouting complete: {len(decisions)} decisions")

## 8. Production Metrics Analysis

In [ ]:
# Convert decisions to DataFrame
decisions_df = pd.DataFrame([d.to_dict() for d in decisions])

# Add to original data
df['stream'] = decisions_df['stream'].values
df['urgency'] = decisions_df['urgency'].values
df['temperature'] = decisions_df['temperature'].values
df['confidence_score'] = decisions_df['confidence_score'].values
df['p_green'] = decisions_df['p_green'].values
df['p_yellow'] = decisions_df['p_yellow'].values
df['p_red'] = decisions_df['p_red'].values

# Get router metrics
metrics = router.get_metrics()

print("="*60)
print("PRODUCTION ROUTING METRICS")
print("="*60)
print(f"\nTotal Decisions: {metrics['n_decisions']}")
print(f"\nStream Distribution:")
print(f"  🟢 GREEN (Auto):    {metrics['automation_rate']*100:.1f}%")
print(f"  🟡 YELLOW (Assist): {metrics['assisted_rate']*100:.1f}%")
print(f"  🔴 RED (Expert):    {metrics['expert_rate']*100:.1f}%")
print(f"\nQuality Metrics:")
print(f"  Avg Confidence: {metrics['avg_confidence']:.3f}")
print(f"  Avg Temperature: {metrics['avg_temperature']:.3f}")
print(f"  Avg Quality (α): {metrics['avg_quality_alpha']:.3f}")
print(f"\nSpecial Cases:")
print(f"  Knockout Rate: {metrics['knockout_rate']*100:.1f}%")
print(f"  Collapse Rate: {metrics['collapse_rate']*100:.1f}%")

In [ ]:
# Temperature explanation
print(router.explain_temperature())

In [ ]:
# Analyze by category
category_routing = df.groupby('category').agg({
    'stream': lambda x: (x == 'green').mean(),
    'temperature': 'mean',
    'R': 'mean',
    'N': 'mean',
    'amount': 'mean'
}).round(3)
category_routing.columns = ['automation_rate', 'avg_temp', 'avg_R', 'avg_N', 'avg_amount']

print("\nRouting by Category:")
print(category_routing.sort_values('automation_rate', ascending=False))

## 9. Collapse Type Analysis

In [ ]:
# Analyze each collapse type
collapse_analysis = df.groupby('collapse_type').agg({
    'stream': lambda x: pd.Series({'green': (x=='green').mean(), 
                                   'yellow': (x=='yellow').mean(),
                                   'red': (x=='red').mean()}).to_dict(),
    'R': 'mean',
    'S': 'mean',
    'N': 'mean',
    'temperature': 'mean'
})

print("\nCollapse Type Analysis:")
print("="*60)
for collapse_type in df['collapse_type'].unique():
    subset = df[df['collapse_type'] == collapse_type]
    print(f"\n{collapse_type.upper()}:")
    print(f"  Count: {len(subset)}")
    print(f"  Avg YRSN: R={subset['R'].mean():.2f}, S={subset['S'].mean():.2f}, N={subset['N'].mean():.2f}")
    print(f"  Avg τ: {subset['temperature'].mean():.2f}")
    print(f"  Stream: GREEN={100*(subset['stream']=='green').mean():.0f}%, "
          f"YELLOW={100*(subset['stream']=='yellow').mean():.0f}%, "
          f"RED={100*(subset['stream']=='red').mean():.0f}%")

## 10. Urgency Distribution

In [ ]:
# Urgency analysis
print("\nUrgency Distribution:")
print(df['urgency'].value_counts())

# Urgency by stream
print("\nUrgency by Stream:")
print(pd.crosstab(df['stream'], df['urgency'], normalize='index').round(2))

## 11. Sample Routing Decisions

In [ ]:
# Show sample decisions from each stream
for stream in ['green', 'yellow', 'red']:
    print(f"\n{'='*60}")
    print(f"{stream.upper()} STREAM SAMPLES")
    print(f"{'='*60}")
    
    samples = df[df['stream'] == stream].head(3)
    for _, row in samples.iterrows():
        print(f"\n[{row['request_id']}] {row['text'][:60]}...")
        print(f"  Category: {row['category']}, Amount: ${row['amount']:,.0f}")
        print(f"  YRSN: R={row['R']:.2f}, S={row['S']:.2f}, N={row['N']:.2f}")
        print(f"  Temperature: τ={row['temperature']:.2f}")
        print(f"  Confidence: {row['confidence_score']:.3f}")
        print(f"  Urgency: {row['urgency']}")
        print(f"  Soft Probs: G={row['p_green']:.2f}, Y={row['p_yellow']:.2f}, R={row['p_red']:.2f}")

## 12. Export Production Results

In [ ]:
# Export routing decisions
output_cols = ['request_id', 'text', 'category', 'amount', 'decision',
               'R', 'S', 'N', 'collapse_type', 'temperature', 
               'confidence_score', 'stream', 'urgency',
               'p_green', 'p_yellow', 'p_red']

df[output_cols].to_csv('production_routing_results.csv', index=False)

# Export metrics summary
metrics_summary = {
    **metrics,
    'dataset_health': health_score,
    'n_categories': df['category'].nunique(),
    'temperature_mode': 'ANNEALING',
    'timestamp': datetime.now().isoformat()
}
pd.DataFrame([metrics_summary]).to_csv('production_metrics.csv', index=False)

print(f"Exported {len(df)} routing decisions")
print(f"Exported production metrics")

## Summary

In this notebook we:
1. Generated a production-scale dataset (2000+ requests across 10 categories)
2. Trained a classifier with cross-validation
3. Used Cleanlab Datalab for comprehensive issue detection
4. Converted all signals to YRSN via CleanlabAdapter
5. Initialized ApprovalRouter with ANNEALING temperature schedule
6. Processed all requests and analyzed production metrics
7. Examined routing patterns by category and collapse type
8. Analyzed urgency distribution across streams

**Key Insights**:
- Temperature annealing allows gradual transition from conservative to automated routing
- Different categories have different natural automation rates based on data quality
- Collapse types (POISONING, CONFUSION) properly route to RED stream
- The τ = 1/α relationship ensures system self-calibrates based on quality

---

## Production Deployment Checklist

- [ ] Configure AWS Bedrock for embeddings
- [ ] Set up SageMaker endpoint for classifier
- [ ] Deploy Datalab analysis as Lambda function
- [ ] Configure DynamoDB for decision audit trail
- [ ] Set up CloudWatch metrics for monitoring
- [ ] Configure SNS alerts for high collapse rates
- [ ] Implement A/B testing infrastructure
- [ ] Set up periodic model retraining pipeline